In [1]:
import pandas as pd
import numpy as np
from nltk import tag, word_tokenize
from time import time
from os import path, makedirs
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.externals import joblib
from utility import normalize_data

In [2]:
## load data
df1 = pd.read_csv('data/S08/question_answer_pairs.txt', sep='\\t', engine='python').dropna()
df2 = pd.read_csv('data/S09/question_answer_pairs.txt', sep='\\t', engine='python').dropna()
df3 = pd.read_csv('data/S10/question_answer_pairs.txt', sep='\\t', engine='python').dropna()
frames = [df1, df2, df3]
df = pd.concat(frames)

questions = df['Question']
answers = df['Answer']

In [3]:
##
## categorization of penn-tree
##
## - https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
##
penn_scale = {
    'CC': 1,
    'CD': 2,
    'DT': 3, 
    'EX': 4,
    'FW': 5,
    'IN': 6,
    'JJ': 7,
    'JJR': 8,
    'JJS': 9,
    'LS': 10,
    'MD': 11,
    'NN': 12,
    'NNS': 13,
    'NNP': 14,
    'NNPS': 15,
    'PDT': 16,
    'POS': 17,
    'PRP': 18,
    'PRP$': 19,
    'RB': 20,
    'RBR': 21,
    'RBS': 22,
    'RP': 23,
    'SYM': 24,
    'TO': 25,
    'UH': 26,
    'VB': 27,
    'VBD': 28,
    'VBG': 29,
    'VBN': 30,
    'VBP': 31,
    'VBZ': 32,
    'WDT': 33,
    'WP': 34,
    'WP$': 35,
    'WRB': 36
}

def tokenizer(sentence):
    '''

    penn-tree: tokenize + parts of speech

    '''

    sent = word_tokenize(sentence)
    pos = tag.pos_tag(sent)
    return([x[1] for x in pos if x[1] and x[1] in penn_scale])

def replace(list, dictionary):
    '''

    replace list item with corresponding dict value

    '''

    return [dictionary.get(item, item) for item in list]

In [ ]:
##
## order of appending 'pos' (1) must match the order of appending
##     the 'sent_type' (2) sentence type.
##

## (1) pos: combine questions + answers
questions_pos = questions.apply(tokenizer)
questions_unique = [replace(list(x), penn_scale)  for x in set(tuple(x) for x in questions_pos)]

answers_pos = answers.apply(tokenizer)
answers_unique = [replace(list(x), penn_scale) for x in set(tuple(x) for x in answers_pos)]

## append data
pos = answers_unique + questions_unique

## (2) sentence type
sent_type = []
for i in range(len(questions_unique)):
    sent_type.append('0')

for j in range(len(answers_unique)):
    sent_type.append('1')

In [ ]:
## adjusted dataset
df_adjusted = pd.DataFrame({
    'pos': pos,
    'type': sent_type
})

## train + test
X_train, X_test, y_train, y_test = train_test_split(
    df_adjusted['pos'],
    df_adjusted['type'],
    test_size=0.2
)

## print shape
print('X_train: {X}, y_train: {y}'.format(
    X=X_train.shape,
    y=y_train.shape
))

print('X_test: {X}, y_test: {y}'.format(
    X=X_test.shape,
    y=y_test.shape
))

In [ ]:
X_train.head(10)

In [ ]:
y_train.head(10)

In [ ]:
## stop gap: used for column padding
stop_gap = 40

## train: ensure nested lists same length
length = len(sorted(X_train, key=len, reverse=True)[0])
X_train_final=np.array([xi+[stop_gap]*(length-len(xi)) for xi in X_train])
X_train_final=pd.DataFrame(X_train_final)

## test: ensure nested lists same length
length = len(sorted(X_test, key=len, reverse=True)[0])
X_test_final=np.array([xi+[stop_gap]*(length-len(xi)) for xi in X_test])
X_test_final=pd.DataFrame(X_test_final)

In [ ]:
## normalize datasets
X_train_final, X_test_final = normalize_data(X_train_final, X_test_final, stop_gap=stop_gap)

In [ ]:
print(X_train_final.head(10))
print(X_test_final.head(10))

In [ ]:
##
## random forrest: selected due to high accuracy, and hasn't
##     been implemented in the project.
##
clf=RandomForestClassifier(n_estimators=1000)

tr0 = time()
clf.fit(X_train_final, np.asarray(y_train))
tr1 = time()
y_pred=clf.predict(X_test_final)
tr2 = time()

In [ ]:
confusion_matrix(y_pred=y_pred, y_true=y_test)

In [ ]:
print('Accuracy: {accuracy}'.format(accuracy=accuracy_score(y_test, y_pred)))

In [ ]:
print('random forrest\ntrain: {rf_train},\npredict: {rf_predict}'.format(
    rf_train=tr1-tr0,
    rf_predict=tr2-tr1
))

In [ ]:
## export model
if not path.exists('model'):
    makedirs('model')
joblib.dump(clf, 'model/random_forrest.pkl', compress=9)